In [1]:
#hp = hyperparameters[dataset]

#classes = hp['classes']

import pandas as pd
#import xlrd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.impute import KNNImputer
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
pd.DataFrame.iteritems=pd.DataFrame.items
# change the strings to ints]
#input= pd.DataFrame(pd.read_csv("US Insurance Claims Data.xlsx", encoding='utf-16-le', encoding_errors='ignore'))
input=pd.read_excel("US_Insurance_Claims_Data.xlsx")
input=input.fillna(method='bfill').fillna(method='ffill')

In [2]:
input.incident_location=input.incident_location.astype('category').cat.codes
input.policy_state=input.policy_state.astype('category').cat.codes
input.insured_education_level=input.insured_education_level.astype('category').cat.codes
input.insured_sex=input.insured_sex.astype('category').cat.codes
input.insured_hobbies=input.insured_hobbies.astype('category').cat.codes
input.insured_relationship=input.insured_relationship.astype('category').cat.codes
input.incident_type=input.incident_type.astype('category').cat.codes
input.collision_type=input.collision_type.astype('category').cat.codes
input.incident_severity=input.incident_severity.astype('category').cat.codes
input.authorities_contacted=input.authorities_contacted.astype('category').cat.codes
input.incident_state=input.incident_state.astype('category').cat.codes
input.incident_city=input.incident_city.astype('category').cat.codes
input.insured_occupation=input.insured_occupation.astype('category').cat.codes
input.property_damage=input.property_damage.astype('category').cat.codes
input.police_report_available=input.police_report_available.astype('category').cat.codes
input.auto_make=input.auto_make.astype('category').cat.codes
input.auto_model=input.auto_model.astype('category').cat.codes
#input['fraud_reported']=input['fraud_reported'].fillna(method='ffill').fillna(method='bfill')
input.fraud_reported=input.fraud_reported.astype('category').cat.codes
input.incident_date=input.incident_date.astype('category').cat.codes
input.policy_bind_date=input.incident_date.astype('category').cat.codes
input['policy_csl'] = input['policy_csl'].apply(lambda x: eval(x))

In [3]:
#img_rows, img_cols = 32, 32
#train_size = trainX.shape[0]

def KNN_impute(input, k):
    imputer = KNNImputer(n_neighbors=k)
    df_filled = imputer.fit_transform(input)
    matrix=pd.DataFrame(df_filled, columns=['months_as_customer', 'age', 'policy_number', 'policy_bind_date' , 'policy_state', 'policy_csl', 'policy_deductable', 'policy_annual_premium', 'umbrella_limit', 'insured_zip', 'insured_sex', 'insured_education_level', 'insured_occupation', 'insured_hobbies', 'insured_relationship', 'capital-gains', 'capital-loss', 'incident_date', 'incident_type', 'collision_type', 'incident_severity', 'authorities_contacted', 'incident_state', 'incident_city', 'incident_location', 'incident_hour_of_the_day', 'number_of_vehicles_involved', 'property_damage', 'bodily_injuries', 'witnesses', 'police_report_available', 'total_claim_amount', 'injury_claim', 'property_claim', 'vehicle_claim', 'auto_make', 'auto_model', 'auto_year', 'fraud_reported'])
    #matrix = pd.DataFrame(df_filled, columns=['months_as_customer', 'age', 'policy_number', 'policy_bind_date', 'policy_state', 'policy_csl', 'policy_deductable', 'policy_annual_premium', 'insured_zip', 'insured_sex', 'insured_education_level', 'insured_occupation', 'insured_hobbies', 'insured_relationship', 'capital-gains', 'capital-loss', 'incident_date', ])
    matrix.to_csv("DataKNN" +  str(k) + ".csv" )
    return matrix

for k in range(1,20,1):
    matrix=KNN_impute(input, k)
    aws=KNeighborsRegressor(k)
    X = matrix.drop('fraud_reported',axis=1)
    y =  matrix['fraud_reported']
    from imblearn.under_sampling import RandomUnderSampler

    ros = RandomUnderSampler()
    X_res, y_res = ros.fit_resample(X, y)
    aws.fit(X_res, y_res)
    cv_scores = cross_val_score(aws, X_res, y_res,  cv=10, scoring='neg_mean_squared_error')
    print(tf.math.abs(cv_scores.mean()))

tf.Tensor(0.25455432859496063, shape=(), dtype=float64)
tf.Tensor(0.22527721005429807, shape=(), dtype=float64)
tf.Tensor(0.22981096220373876, shape=(), dtype=float64)
tf.Tensor(0.2456511322270351, shape=(), dtype=float64)
tf.Tensor(0.2630404693632685, shape=(), dtype=float64)
tf.Tensor(0.28240764748947594, shape=(), dtype=float64)
tf.Tensor(0.2838972627669822, shape=(), dtype=float64)
tf.Tensor(0.296305685093954, shape=(), dtype=float64)
tf.Tensor(0.30167030587145965, shape=(), dtype=float64)
tf.Tensor(0.30485903341264514, shape=(), dtype=float64)
tf.Tensor(0.3125306284961528, shape=(), dtype=float64)
tf.Tensor(0.30535246377867703, shape=(), dtype=float64)
tf.Tensor(0.3137295603001454, shape=(), dtype=float64)
tf.Tensor(0.31763062585781066, shape=(), dtype=float64)
tf.Tensor(0.31813805905228026, shape=(), dtype=float64)
tf.Tensor(0.32051290453475484, shape=(), dtype=float64)
tf.Tensor(0.31768172387904936, shape=(), dtype=float64)
tf.Tensor(0.32168594007852847, shape=(), dtype=float64)

In [4]:
import tensorflow as tf

In [37]:
# trainX,testX,trainY,testY=train_test_split(X_res,y_res,test_size=0.2)
matrix=KNN_impute(input, 2)   
from tensorflow.keras.preprocessing.sequence import pad_sequences

#trainX_padded = pad_sequences(trainX, padding='post', truncating='post')
#testX_padded = pad_sequences(testX, padding='post', truncating='post')
#trainY_padded = pad_sequences(trainY, padding='post', truncating='post')
#testY_padded = pad_sequences(testY, padding='post', truncating='post')



grouped = input.groupby('months_as_customer')

# Extract sequences and labels
sequences = grouped.apply(lambda X: X[['months_as_customer', 'age', 'policy_number', 'policy_bind_date' , 'policy_state', 'policy_csl', 'policy_deductable', 'policy_annual_premium', 'umbrella_limit', 'insured_zip', 'insured_sex', 'insured_education_level', 'insured_occupation', 'insured_hobbies', 'insured_relationship', 'capital-gains', 'capital-loss', 'incident_date', 'incident_type', 'collision_type', 'incident_severity', 'authorities_contacted', 'incident_state', 'incident_city', 'incident_location', 'incident_hour_of_the_day', 'number_of_vehicles_involved', 'property_damage', 'bodily_injuries', 'witnesses', 'police_report_available', 'total_claim_amount', 'injury_claim', 'property_claim', 'vehicle_claim', 'auto_make', 'auto_model', 'auto_year']].values.tolist())
labels = grouped['fraud_reported'].first().values
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Pad to same length (post-padding with zeros)
X = pad_sequences(sequences.tolist(), padding='post', dtype='float32')
y = np.array(labels)




from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

# Flatten for oversampling

#X_resampled, y_resampled = ros.fit_resample(X_flat, y)
#
#X_resampled, y_resampled = sm.fit_resample(X_flat, y)


# Reshape back to 3D

X_flat = X.reshape((X.shape[0], -1))
#ros = RandomOverSampler(random_state=1)
ros=RandomUnderSampler(random_state=1000)
sm=SMOTE(random_state=1000)

X_resampled, y_resampled = sm.fit_resample(X_flat, y)
X_resampled, y_resampled = ros.fit_resample(X_resampled,  y_resampled)

X_resampled = X_resampled.reshape((-1, X.shape[1], X.shape[2]))

trainX,testX,trainY,testY=train_test_split((X_resampled), (y_resampled), test_size=0.2, random_state=1000)


#trainY = utils.to_categorical(trainY)
#testY = utils.to_categorical(testY)
#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'top_k_categorical_accuracy'])
#csv_logger = CSVLogger(logfile, append=True, separator=';')





In [68]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Masking
from sklearn.utils import class_weight
from tensorflow.keras.initializers import RandomNormal

optim_adam = tf.compat.v1.train.AdamOptimizer(0.1)

model = Sequential([
    Masking(mask_value=0.0, input_shape=(X.shape[1], X.shape[2])),
    #LSTM(2),
    LSTM(128), # Bad init,
   # Dense(1, activation='relu'),
    # Binary classification
    #kernel_initializer='random_normal'),
    Dense(1, activation='sigmoid')  # Binary classification
    
])


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Masking
from sklearn.utils import class_weight

optim_adam = tf.compat.v1.train.AdamOptimizer(0.1)







convex_loss_fn = tf.keras.losses.BinaryFocalCrossentropy(
    apply_class_balancing=False, alpha=0.25, gamma=0.0, from_logits=False,
    label_smoothing=0.0, axis=-1, reduction='sum_over_batch_size'
)


hybrid_convex_loss_fn = tf.keras.losses.BinaryFocalCrossentropy(
    apply_class_balancing=False, alpha=0.25, gamma=2.0, from_logits=False,
    label_smoothing=0.0, axis=-1, reduction='sum_over_batch_size'
)


nonconvex_loss_fn = tf.keras.losses.BinaryFocalCrossentropy(
    apply_class_balancing=False, alpha=0.25, gamma=8.0, from_logits=False,
    label_smoothing=0.0, axis=-1, reduction='sum_over_batch_size'
)


import tensorflow as tf
import tensorflow.keras.backend as K

class NoisyLossWrapper(tf.keras.losses.Loss):
    def __init__(self, base_loss_fn, noise_scale=0.5, name='noisy_loss'):
        super(NoisyLossWrapper, self).__init__(name=name)
        self.base_loss_fn = base_loss_fn
        self.noise_scale = noise_scale

    def call(self, y_true, y_pred):
        base_loss = self.base_loss_fn(y_true, y_pred)
        noise = tf.random.normal(shape=tf.shape(base_loss), mean=0.0, stddev=self.noise_scale)
        return base_loss + noise


nonconvex_focal_loss = NoisyLossWrapper(nonconvex_loss_fn, noise_scale=0.5)
convex_focal_loss = NoisyLossWrapper(convex_loss_fn, noise_scale=0.5)

#nonconvex_focal_loss = nonconvex_loss_fn
#convex_focal_loss = convex_loss_fn

In [81]:

initial_learning_rate = 0.0001

# You can adjust decay_steps and decay_rate based on your dataset size
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=1000,     # Number of steps before decay
    decay_rate=0.9,       # Decay multiplier
    staircase=True        # Decay in steps instead of smoothly
)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

training_schedule = [
    {"loss_fn": convex_focal_loss, "epochs": 5},      # Phase 1: convex loss recommended 5 epochs
    {"loss_fn": nonconvex_focal_loss, "epochs": 95},  # Phase 2: nonconvex loss
   # {"loss_fn": convex_focal_loss, "epochs": 15},     # Phase 3: convex again (optional)
   # {"loss_fn": convex_focal_loss, "epochs": 50}   # Phase 4: final stage
]

# Compile the model before training
current_epoch = 0

for i, stage in enumerate(training_schedule):
    print(f"\n🔁 Training Stage {i + 1} | Epochs: {stage['epochs']} | Loss: {stage['loss_fn'].name}")
    
    model.compile(
        optimizer=optimizer,
        loss=stage['loss_fn'],
        metrics=['accuracy']
    )
    
    model.fit(
        trainX, trainY,
        initial_epoch=current_epoch,
        epochs=current_epoch + stage["epochs"],
        batch_size=1,
        validation_data=(testX, testY),
        verbose=1
    )
    
    current_epoch += stage["epochs"]



🔁 Training Stage 1 | Epochs: 5 | Loss: noisy_loss
Epoch 1/5
633/633 [==============================] - 6s 8ms/step - loss: 0.5979 - accuracy: 0.6856 - val_loss: 0.6141 - val_accuracy: 0.7296
Epoch 2/5
633/633 [==============================] - 5s 7ms/step - loss: 0.5952 - accuracy: 0.6888 - val_loss: 0.6564 - val_accuracy: 0.6478
Epoch 3/5
633/633 [==============================] - 4s 7ms/step - loss: 0.6216 - accuracy: 0.6840 - val_loss: 0.5979 - val_accuracy: 0.6792
Epoch 4/5
633/633 [==============================] - 4s 7ms/step - loss: 0.6185 - accuracy: 0.6983 - val_loss: 0.6343 - val_accuracy: 0.6730
Epoch 5/5
633/633 [==============================] - 4s 7ms/step - loss: 0.5957 - accuracy: 0.6872 - val_loss: 0.5566 - val_accuracy: 0.6855



🔁 Training Stage 2 | Epochs: 95 | Loss: noisy_loss
Epoch 6/100
633/633 [==============================] - 6s 8ms/step - loss: -0.0073 - accuracy: 0.6935 - val_loss: -0.0692 - val_accuracy: 0.7044
Epoch 7/100
633/633 [==============================] - 5s 7ms/step - loss: 0.0200 - accuracy: 0.7046 - val_loss: 0.0127 - val_accuracy: 0.7233
Epoch 8/100
633/633 [==============================] - 5s 7ms/step - loss: 0.0079 - accuracy: 0.6951 - val_loss: -0.0389 - val_accuracy: 0.6981
Epoch 9/100
633/633 [==============================] - 5s 7ms/step - loss: 0.0189 - accuracy: 0.6983 - val_loss: 0.0050 - val_accuracy: 0.7107
Epoch 10/100
633/633 [==============================] - 5s 7ms/step - loss: 0.0023 - accuracy: 0.7030 - val_loss: -0.0700 - val_accuracy: 0.7610
Epoch 11/100
633/633 [==============================] - 5s 7ms/step - loss: 0.0166 - accuracy: 0.7014 - val_loss: -0.0129 - val_accuracy: 0.7673
Epoch 12/100
633/633 [==============================] - 5s 7ms/step - loss: -0.0022

pwd